<a href="https://colab.research.google.com/github/hailusong/nlp-qa/blob/master/nlp-seq-classification-captum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sequence Classification Interpretation
Sources
- https://captum.ai/tutorials/IMDB_TorchText_Interpret

### Install modules

In [1]:
!pip install transformers

     |████████████████████████████████| 778kB 2.6MB/s 
     |████████████████████████████████| 890kB 11.0MB/s 
     |████████████████████████████████| 1.1MB 18.0MB/s 
     |████████████████████████████████| 3.0MB 27.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=d4ed8906199b14bb6593e4fe3ad8ee1c5ae2760fa06dc1ba9963193b9e7237f6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


### Load pre-trained sentimental classification model from zoo

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [3]:
print(model)
print(tokenizer)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

### Inference

In [4]:
import torch
import numpy as np

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

cuda:0


In [6]:
sent = "Hello, my dog is awful"

# input_ids = torch.tensor(tokenizer.encode(sent, add_special_tokens=True)) ## .unsqueeze(0)  # Batch size 1
# outputs = model(input_ids, labels=input_ids)

encoded_dict = tokenizer.encode_plus(
    sent,
    add_special_tokens = True,
    max_length = 256,
    pad_to_max_length = False,
    return_atention_mask = False,
    return_tensors = 'pt',  # return pytorch tensors, not tensorflow
    )

# make it a list
# input_ids = [].append(encoded_dict['input_ids'])
input_ids = encoded_dict['input_ids']

# make it a torch tensor and load to CUDA
# input_ids = torch.cat(input_ids, dim=0)
b_input_ids = input_ids.to(device)

with torch.no_grad():
    logits, = model(b_input_ids)

probs = torch.nn.functional.softmax(logits, dim=1)

# also convert to numpy so we can use some numpy functions
logits = logits.cpu().numpy()

# get the index of the item with max logit value (probability)
pred_flat = np.argmax(logits, axis=1).flatten()[0]

# and then get the probability -> confidence
confidence = int(probs.flatten()[pred_flat]*100)

print(f'Prediction: {pred_flat} with confidnece {confidence}')

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Keyword arguments {'return_atention_mask': False} not recognized.


Prediction: 0 with confidnece 99


### Interpretation

In [7]:
!pip install captum
!pip install spacy

     |████████████████████████████████| 1.4MB 2.7MB/s 


In [8]:
import spacy
import torchtext
from torchtext.vocab import Vocab
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

nlp = spacy.load('en')

#### Common
Source: http://anie.me/On-Torchtext/<br>
**Torchtext** is a very powerful library that solves the preprocessing of text very well, but we need to know what it can and can’t do, and understand how each API is mapped to our inherent understanding of what should be done. An additional perk is that Torchtext is designed in a way that it does not just work with PyTorch, but with any deep learning library (for example: Tensorflow).

Let’s compile a list of tasks that text preprocessing must be able to handle. All checked boxes are functionalities provided by Torchtext.

- **Train/Val/Test Split**: seperate your data into a fixed train/val/test set (not used for k-fold validation)
- **File Loading**: load in the corpus from various formats
- **Tokenization**: break sentences into list of words
- **Vocab**: generate a vocabulary list
- **Numericalize/Indexify**: Map words into integer numbers for the entire corpus
- **Word Vector**: either initialize vocabulary randomly or load in from a pretrained embedding, this embedding must be “trimmed”, meaning we only store words in our vocabulary into memory.
- **Batching**: generate batches of training sample (padding is normally happening here)
- **Embedding Lookup**: map each sentence (which contains word indices) to fixed dimension word vectors

In [9]:
# TEXT = torchtext.data.Field(lower=True, tokenize='spacy')
# Label = torchtext.data.LabelField(dtype = torch.float)

#### Setup baseline for IG

In [10]:
# PAD_IND = TEXT.vocab.stoi['pad']
PAD_IND = tokenizer.pad_token_id
token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)

#### FIxing no target error

In [14]:
def seq_classification_forward_func(input_ids):
    for i in range(0, input_ids.shape[0]):
      print(f'>>{i}>>' + tokenizer.decode(input_ids[i]))

    logits, = model(input_ids)

    # pred = predict(inputs,
    #                token_type_ids=token_type_ids,
    #                position_ids=position_ids,
    #                attention_mask=attention_mask)
    # pred = pred[position]
    print(logits[0].shape)

    # [1, 2] -> [2]
    return logits[0]

#### IG algorithm

In [17]:
import transformers

assert type(model) == transformers.modeling_distilbert.DistilBertForSequenceClassification

# print(type(model.distilbert.embeddings))
# lig = LayerIntegratedGradients(model, model.distilbert.embeddings)
lig = LayerIntegratedGradients(seq_classification_forward_func, model.distilbert.embeddings)

# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

def interpret_sentence(model, sentence, min_len = 7, label = 0):
    # text = [tok.text for tok in nlp.tokenizer(sentence)]
    # if len(text) < min_len:
    #     text += ['pad'] * (min_len - len(text))
    # indexed = [TEXT.vocab.stoi[t] for t in text]

    # model.zero_grad()

    # input_indices = torch.tensor(indexed, device=device)
    # input_indices = input_indices.unsqueeze(0)
    
    # # input_indices dim: [sequence_length]
    # seq_length = min_len

    # # predict
    # pred = forward_with_sigmoid(input_indices).item()
    # pred_ind = round(pred)

    encoded_dict = tokenizer.encode_plus(
        sentence,
        add_special_tokens = False,
        max_length = 256,
        pad_to_max_length = False,
        return_attention_mask = False,
        return_tensors = 'pt',  # return pytorch tensors, not tensorflow
        )

    # make it a list
    # input_ids = [].append(encoded_dict['input_ids'])
    input_ids = encoded_dict['input_ids']
    seq_length = len(input_ids[0])
    print(f'seq_length: {seq_length}')

    text = tokenizer.convert_ids_to_tokens(input_ids[0])
    assert len(text) == len(input_ids[0])

    # make it a torch tensor and load to CUDA
    # input_ids = torch.cat(input_ids, dim=0)
    b_input_ids = input_ids.to(device)

    with torch.no_grad():
        logits, = model(b_input_ids)

    probs = torch.nn.functional.softmax(logits, dim=1)

    # also convert to numpy so we can use some numpy functions
    logits = logits.cpu().numpy()

    # get the index of the item with max logit value (probability)
    pred_flat = np.argmax(logits, axis=1).flatten()[0]
    pred_ind = pred_flat
    print(f'pred_ind: {pred_ind}')

    # and then get the probability -> confidence
    confidence = int(probs.flatten()[pred_flat]*100)

    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)

    # compute attributions and approximation delta using layer integrated gradients
    input_indices = b_input_ids
    print(input_indices)
    print(reference_indices)
    attributions_ig, delta = lig.attribute(input_indices, reference_indices, \
                                           n_steps=50, return_convergence_delta=True)

    print('pred: ', pred_ind, '(', '%.2f'%confidence, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, text, confidence, pred_ind, label, delta, vis_data_records_ig)
    
def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            'positive' if pred_ind == 1 else 'negative',
                            'positive' if label == 1 else 'negative',
                            'positive',
                            attributions.sum(),       
                            text,
                            delta))

#### Interpretation

In [18]:
interpret_sentence(model, 'It was a fantastic performance !', label=1)
# interpret_sentence(model, 'Best film ever', label=1)
# interpret_sentence(model, 'Such a great show!', label=1)
# interpret_sentence(model, 'It was a horrible movie', label=0)
# interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
# interpret_sentence(model, 'It is a disgusting movie!', label=0)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


seq_length: 6
pred_ind: 1
tensor([[ 2009,  2001,  1037, 10392,  2836,   999]], device='cuda:0')
tensor([[0, 0, 0, 0, 0, 0]], device='cuda:0')
>>0>>it was a fantastic performance!
torch.Size([2])
>>0>>[PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
torch.Size([2])
>>0>>it was a fantastic performance!
>>1>>it was a fantastic performance!
>>2>>it was a fantastic performance!
>>3>>it was a fantastic performance!
>>4>>it was a fantastic performance!
>>5>>it was a fantastic performance!
>>6>>it was a fantastic performance!
>>7>>it was a fantastic performance!
>>8>>it was a fantastic performance!
>>9>>it was a fantastic performance!
>>10>>it was a fantastic performance!
>>11>>it was a fantastic performance!
>>12>>it was a fantastic performance!
>>13>>it was a fantastic performance!
>>14>>it was a fantastic performance!
>>15>>it was a fantastic performance!
>>16>>it was a fantastic performance!
>>17>>it was a fantastic performance!
>>18>>it was a fantastic performance!
>>19>>it was a fantastic performance

In [19]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
positive,positive (99.00),positive,-0.71,it was a fantastic performance !
